# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# DATA FILES
path_weather_data = "../weatherpy/output.csv"

# READ CRIME DATA
weather_data = pd.read_csv(path_weather_data)

del weather_data["Unnamed: 0"]

weather_data

,city,latitude,longitude,temperature_farenheit,humidity_percent,cloudiness_percent,windspeed_mph
0,woodstock,43.13,-80.75,65.41,100,75,6.93
1,faanui,-16.48,-151.75,77.59,77,2,8.21
2,vaini,-21.20,-175.20,75.20,88,75,5.82
3,puerto ayora,-0.74,-90.35,66.00,93,46,1.01
4,tuatapere,-46.13,167.68,39.00,83,8,3.00
...,...,...,...,...,...,...,...
555,inza,53.85,46.36,71.31,57,57,6.87
556,marktheidenfeld,49.84,9.60,64.44,72,25,3.02
557,fortuna,40.60,-124.16,54.91,100,90,4.70
558,urumqi,43.80,87.60,96.80,15,0,8.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key from file
gmaps.configure(api_key=g_key)

In [4]:
# Store positions  
positions = weather_data[["latitude", "longitude"]].astype(float)

humidity = weather_data["humidity_percent"].astype(float)

In [5]:
# Create heatmap

fig = gmaps.figure(zoom_level=1,center=[0,0])
fig
heat_layer = gmaps.heatmap_layer(positions, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the DataFrame to find ideal weather condition. 
#A max temperature lower than 85 degrees but higher than 70,wind speed less than 7 mph, less than 30 percent cloudiness.
#save as hotel_data

hotel_df=weather_data.loc[((weather_data["temperature_farenheit"] < 85) 
                               & (weather_data["temperature_farenheit"] > 70) 
                               & (weather_data["windspeed_mph"] < 5)
                               & (weather_data["cloudiness_percent"] < 15)),:]
hotel_data=pd.DataFrame(hotel_df)
hotel_data.head()

,city,latitude,longitude,temperature_farenheit,humidity_percent,cloudiness_percent,windspeed_mph
59,east london,-33.02,27.91,71.60,19,0,2.24
126,manubul,5.47,120.80,83.57,67,7,4.03
144,gat,31.61,34.76,84.81,53,0,1.99
159,vostok,46.49,135.88,70.83,85,0,2.57
178,sayanogorsk,53.09,91.40,80.37,46,14,4.63


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_data['hotel_name'] = ""
hotel_data['country'] = ""
hotel_data

,city,latitude,longitude,temperature_farenheit,humidity_percent,cloudiness_percent,windspeed_mph,hotel_name,country
59,east london,-33.02,27.91,71.60,19,0,2.24,,
126,manubul,5.47,120.80,83.57,67,7,4.03,,
144,gat,31.61,34.76,84.81,53,0,1.99,,
159,vostok,46.49,135.88,70.83,85,0,2.57,,
178,sayanogorsk,53.09,91.40,80.37,46,14,4.63,,
188,tevaitoa,-16.78,-151.50,78.04,73,6,4.56,,
215,fulton,33.77,-84.45,73.92,88,1,3.15,,
258,svetlogorsk,52.63,29.73,73.98,54,5,3.85,,
267,panaba,21.28,-88.27,75.74,96,4,3.87,,
403,kosh-agach,50.00,88.67,73.15,34,0,2.21,,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location":"-14.47,132.27",
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
}


#assemble url and make API request
response = requests.get(base_url, params=params).json()
    
#extract results
results = response['results']
results[0]['plus_code']['compound_code']

'G7P9+R4 Katherine NT, Australia'

In [9]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_data.iterrows():

     # get location from df
    lat = row['latitude']
    lng = row['longitude']

#add location keyword to params dict
    params['location'] = str(lat)+","+str(lng)
    
#assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
#extract results
    results = response['results']
    
    try:      
        hotel_data.loc[index, 'hotel_name'] = results[0]['name']
        hotel_data.loc[index, 'country'] = results[0]['plus_code']['compound_code']
     
    except (KeyError, IndexError):
        hotel_data.loc[index, 'hotel_name'] = "NaN"
        hotel_data.loc[index, 'country'] = "NaN"



In [10]:
#check for new column
hotel_data=hotel_data.loc[(hotel_data["hotel_name"]!='NaN')]
hotel_data

,city,latitude,longitude,temperature_farenheit,humidity_percent,cloudiness_percent,windspeed_mph,hotel_name,country
59,east london,-33.02,27.91,71.60,19,0,2.24,Tu Casa,"2W7F+WC East London, South Africa"
144,gat,31.61,34.76,84.81,53,0,1.99,רגע בנחת,"JPXP+8X Revaha, Israel"
178,sayanogorsk,53.09,91.40,80.37,46,14,4.63,Hotel Sayanogorsk,"4C46+5R Sayanogorsk, Republic of Khakassia, Ru..."
188,tevaitoa,-16.78,-151.50,78.04,73,6,4.56,Hôtel Raiatea Lodge,"7G26+58 Tumaraa, French Polynesia"
215,fulton,33.77,-84.45,73.92,88,1,3.15,Gorgeous City Home!!! Sleeps 10!!,"QH65+PP Atlanta, GA, USA"
258,svetlogorsk,52.63,29.73,73.98,54,5,3.85,Hotel Svetlogorsk JBIK,"JQH6+JR Svietlahorsk, Belarus"
267,panaba,21.28,-88.27,75.74,96,4,3.87,Hotel Amor,"7PRC+Q6 Panabá, Yucatan, Mexico"
403,kosh-agach,50.00,88.67,73.15,34,0,2.21,Fermerskiy Dom v Kosh-Agache,"XJ9Q+GR Kosh-Agach, Altai Republic, Russia"
418,west monroe,32.52,-92.15,72.41,94,1,3.04,Hilton Garden Inn West Monroe,"GR4C+V2 West Monroe, LA, USA"
480,houston,29.76,-95.36,80.83,88,1,4.74,"Hotel ICON, Autograph Collection","QJ7Q+2P Downtown Houston, Houston, TX, USA"


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
locations = hotel_data[["latitude", "longitude"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(
            locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))